<div style="background-color:	#103060; text-align:center; vertical-align: middle; padding:40px 0; margin-top:30px">
<h1 style="color:white">Website Automatisering met WP-Admin API</h1>
<b style="color:white">Rogier Gernaat</b>
    </div>

<a name='start'></a>

In [13]:
!pip install python-wordpress-xmlrpc



  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for python-wordpress-xmlrpc: filename=python_wordpress_xmlrpc-2.3-py3-none-any.whl size=16376 sha256=abc425362c4256903e174e02fe921046379082dba6fb9ef575916dff7879c97e
  Stored in directory: c:\users\rogie\appdata\local\pip\cache\wheels\b9\50\e9\51ec8e2fa457531bcb724c59a5a08ea3bb1df845848a9dad1f
Successfully built python-wordpress-xmlrpc


<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>Inzet in de wijk </strong></h2>
</div>
Dit is code voor het invullen van de inzet in de wijk activiteiten op de correcte pagina

In [ ]:
import os
import mysql.connector
import pandas as pd
from dotenv import load_dotenv
from wordpress_xmlrpc import Client
from wordpress_xmlrpc.methods.posts import EditPost

# Laad Azure-configuratie uit .env
load_dotenv()
db_host = os.getenv('DB_HOST')
db_port = int(os.getenv('DB_PORT', 3306))
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_ssl_ca = os.getenv('DB_SSL_CA')

# WordPress-credentials
wp_url = "https://samenzoetermeergezond.nl/xmlrpc.php"
wp_user = "SamenZoetermeerGezond"
wp_password = "$Lmzk#S1U6gvM5p8&@@Gn2NE"

class AzureDBConnector:
    def __init__(self, host, port, user, password, database, ssl_ca):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.ssl_ca = ssl_ca
        self.connection = None

    def connect(self):
        try:
            self.connection = mysql.connector.connect(
                host=self.host,
                port=self.port,
                user=self.user,
                password=self.password,
                database=self.database,
                ssl_ca=self.ssl_ca
            )
            print("Verbonden met de Azure Database.")
        except mysql.connector.Error as err:
            print(f"Fout bij het verbinden met de database: {err}")

    def fetch_query_data(self, query):
        if not self.connection:
            print("Geen actieve databaseverbinding.")
            return pd.DataFrame()
        try:
            cursor = self.connection.cursor(dictionary=True)
            cursor.execute(query)
            rows = cursor.fetchall()
            df = pd.DataFrame(rows)
            return df
        except Exception as err:
            print(f"Fout bij het uitvoeren van de query: {err}")
            return pd.DataFrame()
        finally:
            cursor.close()

    def close(self):
        if self.connection:
            self.connection.close()
            print("Verbinding met de database gesloten.")

class ActivityPublisher:
    def __init__(self, wp_url, wp_user, wp_password):
        self.client = Client(wp_url, wp_user, wp_password)

    def update_page_with_sidebar(self, page_id, activities, sidebar_links):
        """
        Update de pagina met activiteiten en een zijbalk met links.
        
        :param page_id: ID van de WordPress-pagina.
        :param activities: Pandas DataFrame met activiteiten.
        :param sidebar_links: Dictionary met namen van pagina's en hun URLs.
        """
        # Dynamische afstand berekening
        activity_spacing = max(8, 32 - len(activities))  # Dynamische afstand tussen kaarten

        # HTML voor activiteitenkaarten (gecentreerd)
        content_html = f"<div style='display: flex; flex-wrap: wrap; gap: {activity_spacing}px; justify-content: center; padding: 20px;'>"
        for _, row in activities.iterrows():
            shortened_description = row['Extra_beschrijving'][:150] + '...' if len(row['Extra_beschrijving']) > 150 else row['Extra_beschrijving']
            content_html += f"""
            <div style="width: 280px; border: 1px solid #ddd; border-radius: 8px; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); overflow: hidden; text-align: center;">
                <img src="{row['URL_afbeelding']}" alt="Afbeelding" style="width: 100%; height: auto;">
                <div style="padding: 16px;">
                    <h3 style="font-size: 1.2em; margin-bottom: 8px;">{row['Titel']}</h3>
                    <p style="font-size: 0.9em; color: #555; margin-bottom: 8px;">{shortened_description}</p>
                    <p style="font-size: 0.8em; color: #777; margin-bottom: 16px;"><strong>Locatie:</strong> {row['Locatie']}</p>
                    <a href="{row['Link']}" style="display: inline-block; padding: 10px 16px; background-color: #007BFF; color: white; text-decoration: none; border-radius: 4px;">Ga hier verder</a>
                </div>
            </div>
            """
        content_html += "</div>"

        # HTML voor een moderne zijbalk
        sidebar_html = """
        <div style="padding: 16px; border: 1px solid #ddd; border-radius: 8px; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); margin-top: 20px;">
            <h3 style="font-size: 1.2em; margin-bottom: 16px;">Andere pagina's</h3>
            <div style="display: flex; flex-direction: column; gap: 4px;">
        """
        for name, url in sidebar_links.items():
            sidebar_html += f"""
            <a href="{url}" style="padding: 6px; text-align: left; background-color: #f8f9fa; border-radius: 4px; text-decoration: none; color: #007BFF; font-weight: bold; box-shadow: 1px 1px 5px rgba(0,0,0,0.1);">
                {name}
            </a>
            """
        sidebar_html += "</div></div>"

        # Combineer de activiteiten en zijbalk
        full_html = f"""
        <div style="display: flex; justify-content: space-between; align-items: flex-start; padding: 20px;">
            <div style="flex: 3;">{content_html}</div>
            <div style="flex: 1; margin-left: 20px;">{sidebar_html}</div>
        </div>
        """

        try:
            page = {"post_content": full_html}
            self.client.call(EditPost(page_id, page))
            print(f"Pagina met ID {page_id} succesvol geüpdatet.")
        except Exception as err:
            print(f"Fout bij het updaten van de pagina: {err}")

# Gebruik
if __name__ == "__main__":
    # Verbind met de Azure-database
    db_connector = AzureDBConnector(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name,
        ssl_ca=db_ssl_ca
    )
    db_connector.connect()

    # Haal activiteiten op
    query = "SELECT Titel, Link, Beschrijving, Extra_beschrijving, Locatie, URL_afbeelding FROM activiteiten"
    activiteiten = db_connector.fetch_query_data(query)

    # Links naar andere pagina's
    sidebar_links = {
        "ZoeterMeerActief": "https://samenzoetermeergezond.nl/testomgeving-zma-rogier/",
        "Evie.nl": "https://samenzoetermeergezond.nl/testomgeving-evie-rogier/",
        "GGD appstore": "https://samenzoetermeergezond.nl/testomgeving-ggd_appstore-rogier/",
        "Nederland zorgt voor elkaar": "https://samenzoetermeergezond.nl/testomgeving-nlzve-rogier/",
        "Scheidingspunt": "https://samenzoetermeergezond.nl/testomgeving-scheidingspunt-rogier/",
        "UITgenda": "https://samenzoetermeergezond.nl/testomgeving-uitgenda-rogier/",
        "ZoetermeerTegenEenzaamheid": "https://samenzoetermeergezond.nl/testomgeving-zte-rogier/",
        "ZoetermeerVoorElkaar": "https://samenzoetermeergezond.nl/testomgeving-zve-rogier/"
        
    }

    # Verbind met WordPress
    publisher = ActivityPublisher(wp_url, wp_user, wp_password)

    # Update de pagina met activiteiten en zijbalk
    private_page_id = 8074  # Pas aan met de juiste pagina-ID
    publisher.update_page_with_sidebar(private_page_id, activiteiten, sidebar_links)

    # Sluit de databaseverbinding
    db_connector.close()


Verbonden met de Azure Database.
Pagina met ID 8074 succesvol geüpdatet.
Verbinding met de database gesloten.


<a name='2.0'></a>

<div style="background-color:#006096; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>ZoeterMeerActief </strong></h2>
</div>
Dit is code voor het invullen van de ZoeterMeerActief activiteiten op de correcte pagina

In [96]:
import os
import mysql.connector
import pandas as pd
from dotenv import load_dotenv
from wordpress_xmlrpc import Client
from wordpress_xmlrpc.methods.posts import EditPost

# Laad Azure-configuratie uit .env
load_dotenv()
db_host = os.getenv('DB_HOST')
db_port = int(os.getenv('DB_PORT', 3306))
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_ssl_ca = os.getenv('DB_SSL_CA')

# WordPress-credentials
wp_url = "https://samenzoetermeergezond.nl/xmlrpc.php"
wp_user = "SamenZoeterMeerGezond"
wp_password = "$Lmzk#S1U6gvM5p8&@@Gn2NE"

class AzureDBConnector:
    def __init__(self, host, port, user, password, database, ssl_ca):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database
        self.ssl_ca = ssl_ca
        self.connection = None

    def connect(self):
        try:
            self.connection = mysql.connector.connect(
                host=self.host,
                port=self.port,
                user=self.user,
                password=self.password,
                database=self.database,
                ssl_ca=self.ssl_ca
            )
            print("Verbonden met de Azure Database.")
        except mysql.connector.Error as err:
            print(f"Fout bij het verbinden met de database: {err}")

    def fetch_query_data(self, query):
        if not self.connection:
            print("Geen actieve databaseverbinding.")
            return pd.DataFrame()
        try:
            cursor = self.connection.cursor(dictionary=True)
            cursor.execute(query)
            rows = cursor.fetchall()
            df = pd.DataFrame(rows)
            return df
        except Exception as err:
            print(f"Fout bij het uitvoeren van de query: {err}")
            return pd.DataFrame()
        finally:
            cursor.close()

    def close(self):
        if self.connection:
            self.connection.close()
            print("Verbinding met de database gesloten.")

class ActivityPublisher:
    def __init__(self, wp_url, wp_user, wp_password):
        self.client = Client(wp_url, wp_user, wp_password)

    def update_page_with_sidebar(self, page_id, activities, sidebar_links):
        """
        Update de pagina met activiteiten en een zijbalk met links.
        
        :param page_id: ID van de WordPress-pagina.
        :param activities: Pandas DataFrame met activiteiten.
        :param sidebar_links: Dictionary met namen van pagina's en hun URLs.
        """
        # HTML voor activiteitenkaarten (gecentreerd)
        content_html = """
        <style>
            .activity-container {
                display: flex;
                flex-wrap: wrap;
                gap: 16px;
                justify-content: center;
                padding: 20px;
            }
            .activity-card {
                flex: 1 1 calc(25% - 16px); /* Zorg voor 4 kaarten naast elkaar */
                max-width: calc(25% - 16px);
                border: 1px solid #ddd;
                border-radius: 8px;
                box-shadow: 2px 2px 8px rgba(0,0,0,0.1);
                overflow: hidden;
                text-align: center;
                background-color: #fff;
            }
            .activity-card img {
                width: 100%;
                height: auto;
            }
            .activity-card-content {
                padding: 16px;
            }
            .activity-card h3 {
                font-size: 1.2em;
                margin-bottom: 8px;
            }
            .activity-card p {
                font-size: 0.9em;
                color: #555;
                margin-bottom: 8px;
            }
            .activity-card a {
                display: inline-block;
                padding: 10px 16px;
                background-color: #007BFF;
                color: white;
                text-decoration: none;
                border-radius: 4px;
            }
            @media (max-width: 768px) {
                .activity-card {
                    flex: 1 1 calc(50% - 16px); /* 2 kaarten naast elkaar op kleinere schermen */
                    max-width: calc(50% - 16px);
                }
            }
            @media (max-width: 480px) {
                .activity-card {
                    flex: 1 1 100%; /* 1 kaart per rij op mobiele apparaten */
                    max-width: 100%;
                }
            }
        </style>
        <div class='activity-container'>
        """
        for _, row in activities.iterrows():
            image_url = row['URL_afbeelding'] if row['URL_afbeelding'] else row['Url_header_afbeelding']
            shortened_description = row['Extra_beschrijving'][:100] + '...' if len(row['Extra_beschrijving']) > 100 else row['Extra_beschrijving']
            content_html += f"""
            <div class="activity-card">
                <img src="{image_url}" alt="Afbeelding">
                <div class="activity-card-content">
                    <h3>{row['Titel']}</h3>
                    <p>{shortened_description}</p>
                    <p><strong>Startdatum:</strong> {row['Startdatum']}</p>
                    <a href="{row['Link']}">Ga hier verder</a>
                </div>
            </div>
            """
        content_html += "</div>"

        # HTML voor een moderne zijbalk
        sidebar_html = """
        <div style="padding: 16px; border: 1px solid #ddd; border-radius: 8px; box-shadow: 2px 2px 8px rgba(0,0,0,0.1); margin-top: 20px;">
            <h3 style="font-size: 1.2em; margin-bottom: 16px;">Andere pagina's</h3>
            <div style="display: flex; flex-direction: column; gap: 8px;">
        """
        for name, url in sidebar_links.items():
            sidebar_html += f"""
            <a href="{url}" style="padding: 10px; text-align: center; background-color: #f8f9fa; border-radius: 4px; text-decoration: none; color: #007BFF; font-weight: bold; box-shadow: 1px 1px 5px rgba(0,0,0,0.1);">
                {name}
            </a>
            """
        sidebar_html += "</div></div>"

        # Combineer de activiteiten en zijbalk
        full_html = f"""
        <div style="display: flex; justify-content: space-between; align-items: flex-start; padding: 20px;">
            <div style="flex: 3;">{content_html}</div>
            <div style="flex: 1; margin-left: 20px;">{sidebar_html}</div>
        </div>
        """

        try:
            page = {"post_content": full_html}
            self.client.call(EditPost(page_id, page))
            print(f"Pagina met ID {page_id} succesvol geüpdatet.")
        except Exception as err:
            print(f"Fout bij het updaten van de pagina: {err}")

# Gebruik
if __name__ == "__main__":
    # Verbind met de Azure-database
    db_connector = AzureDBConnector(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database=db_name,
        ssl_ca=db_ssl_ca
    )
    db_connector.connect()

    # Haal activiteiten op
    query = "SELECT Titel, Link, Extra_beschrijving, Startdatum, URL_afbeelding, Url_header_afbeelding FROM activiteiten_zma"
    activiteiten = db_connector.fetch_query_data(query)

    # Links naar andere pagina's
    sidebar_links = {
        "Inzet in de wijk": "https://samenzoetermeergezond.nl/testomgeving-rogier/",
        "Evie.nl": "https://samenzoetermeergezond.nl/testomgeving-evie-rogier/",
        "GGD appstore": "https://samenzoetermeergezond.nl/testomgeving-ggd_appstore-rogier/",
        "Nederland zorgt voor elkaar": "https://samenzoetermeergezond.nl/testomgeving-nlzve-rogier/",
        "Scheidingspunt": "https://samenzoetermeergezond.nl/testomgeving-scheidingspunt-rogier/",
        "UITgenda": "https://samenzoetermeergezond.nl/testomgeving-uitgenda-rogier/",
        "ZoetermeerTegenEenzaamheid": "https://samenzoetermeergezond.nl/testomgeving-zte-rogier/",
        "ZoetermeerVoorElkaar": "https://samenzoetermeergezond.nl/testomgeving-zve-rogier/"
        
    }

    # Verbind met WordPress
    publisher = ActivityPublisher(wp_url, wp_user, wp_password)

    # Update de pagina met activiteiten en zijbalk
    private_page_id = 8142  # Pas aan met de juiste pagina-ID
    publisher.update_page_with_sidebar(private_page_id, activiteiten, sidebar_links)

    # Sluit de databaseverbinding
    db_connector.close()


Verbonden met de Azure Database.
Pagina met ID 8142 succesvol geüpdatet.
Verbinding met de database gesloten.
